In [1]:
import pandas as pd

In [8]:
iamc_index = ["Region", "Variable", "Unit"]

In [2]:
df = pd.read_csv("C:/Users\maczek\Desktop/emi_drivers.csv")
df = df.set_index(["Region", "Scenario", "Unit"])
df.columns = [int(i) for i in df.columns]

In [5]:
data = pd.read_csv("C:/Users\maczek\Desktop/data.csv").set_index("Region")
data.columns = [int(i) for i in data.columns]

## current implementation

In [7]:
scn_cfg = {}
scn_cfg["region_mapping"] = df.index.get_level_values(0).unique()

In [111]:
old_dfs_dict = {}
new_dfs_dict = {}

In [123]:
for reg in scn_cfg["region_mapping"]:
    #tmp_gr = tmp = self.data.loc[reg, :, driver]
    tmp_gr = tmp = df.loc[reg, :]
    tmp_gr = tmp_gr.pct_change(axis=1)
    for i in range(len(tmp.columns)):
        # 2010 should remain hardcoded - it is the year until
        # which we had calibrated the emi-drivers.
        if tmp.columns[i] <= 2010:
            # If there is no data available, a dataframe with
            # values set to 0 is created.
            if data.empty:
                tmp.loc[:, tmp.columns[i]] = 0
            else:
                tmp.loc[:, tmp.columns[i]] = data.loc[
                reg, tmp.columns[i]
            ]
        else:
            tmp.loc[:, tmp.columns[i]] = (
                tmp.loc[:, tmp.columns[i - 1]]
                + tmp.loc[:, tmp.columns[i - 1]]
                * tmp_gr.loc[:, tmp.columns[i]]
            )
    tmp = tmp.reset_index()
    #tmp["Region"] = reg
    tmp["Unit"] = "Mt CO2/yr"
    #tmp["Variable"] = "test_tec"
    #tmp = tmp.set_index(iamc_index).fillna(0)
    #self.data = tmp.combine_first(self.data)
    old = tmp.set_index(["Scenario", "Unit"])
    old.columns = old.columns.astype(int)
    old_dfs_dict[reg] = old

In [50]:
old_post_2010 = old.drop(old.columns[:4], axis=1).copy(deep=True)

In [134]:
df.loc["R12_CHN", :].divide(df.loc["R12_CHN", :][2010], axis=0).drop(df.loc["R12_CHN", :].columns[:4], axis=1)

,,2010,2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2070,2080,2090,2100,2110
Scenario,Unit,,,,,,,,,,,,,,,,
BIO00GHG000,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO00GHG010,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO00GHG020,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO00GHG050,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO00GHG100,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIO45GHG3000,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO45GHG400,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
BIO45GHG4000,Mt CO2/yr,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


## new vectorized calculation

In [140]:
for reg in scn_cfg["region_mapping"]:
    #tmp = self.data.loc[reg, :, driver]
    tmp = df.loc[reg, :]

    tmp_rel_2010 = tmp.divide(tmp[2010], axis=0).drop(tmp.columns[:4], axis=1)
    tec_afr = data.loc[reg]
    new = tmp_rel_2010[tmp_rel_2010.columns] * tec_afr[2010]
    data_bef_2010 = pd.DataFrame(data.loc[reg, :2010]).T
    for col in data_bef_2010:
        new[col] = data_bef_2010[col].values[0]
    new = new[new.columns.sort_values()]
    new = new.fillna(0)
    
    new_dfs_dict[reg] = new

In [99]:
tmp_rel_2010 = tmp.divide(tmp[2010], axis=0).drop(tmp.columns[:4], axis=1)

In [100]:
tec_afr = data.loc["R12_AFR"]

In [101]:
new = tmp_rel_2010[tmp_rel_2010.columns] * tec_afr[2010]

In [102]:
data_bef_2010 = pd.DataFrame(data.loc[reg, :2010]).T

In [103]:
for col in data_bef_2010:
    new[col] = data_bef_2010[col].values[0]

In [104]:
new = new[new.columns.sort_values()]

In [106]:
new.fillna(0)

,,1990,1995,2000,2005,2010,2015,2020,2025,2030,2035,2040,2045,2050,2055,2060,2070,2080,2090,2100,2110
Scenario,Unit,,,,,,,,,,,,,,,,,,,,
BIO00GHG000,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,104.533321,85.470978,58.128176,31.002874,12.322813,3.466718,0.635540,0.059536,0.0,0.0,0.0,0.0,0.0
BIO00GHG010,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,104.882979,86.171215,59.042042,31.812045,12.809237,3.658228,0.680146,0.064144,0.0,0.0,0.0,0.0,0.0
BIO00GHG020,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,104.987858,86.359776,59.257329,31.975538,12.893656,3.687536,0.686598,0.064881,0.0,0.0,0.0,0.0,0.0
BIO00GHG050,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,105.086286,86.579303,59.559617,32.243910,13.048671,3.744860,0.699316,0.066171,0.0,0.0,0.0,0.0,0.0
BIO00GHG100,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,105.308946,86.940757,59.928075,32.492375,13.163503,3.781908,0.707242,0.067093,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BIO45GHG3000,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,105.317794,86.959742,59.957014,32.522973,13.185622,3.792046,0.710006,0.067277,0.0,0.0,0.0,0.0,0.0
BIO45GHG400,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,105.316503,86.956978,59.952959,32.519102,13.183225,3.791124,0.709638,0.067277,0.0,0.0,0.0,0.0,0.0
BIO45GHG4000,Mt CO2/yr,115.793,115.793,115.793,116.082,116.371,116.862954,117.064786,105.317794,86.959742,59.956645,32.522604,13.185253,3.792046,0.709822,0.067277,0.0,0.0,0.0,0.0,0.0


## check if old and new calculation yield same output

In [107]:
round = 11
(old.fillna(0).round(round) == new.fillna(0).round(round)).all().all()

True

In [142]:
for k,v in old_dfs_dict.items():
    print(k)
    #print(new_dfs_dict[k])
    print((v.fillna(0).round(round) == new_dfs_dict[k].fillna(0).round(round)).all().all())
    if not (v.fillna(0).round(round) == new_dfs_dict[k].fillna(0).round(round)).all().all():
        break

R12_AFR
True
R12_CHN
True
R12_EEU
True
R12_FSU
True
R12_LAM
True
R12_MEA
True
R12_NAM
True
R12_PAO
True
R12_PAS
True
R12_RCPA
True
R12_SAS
True
R12_WEU
True


In [42]:
pd.testing.assert_frame_equal(old, new)